In [9]:
import matplotlib.pyplot as plt
import numpy as np
import pickle
from ramannoodles import spectrafit

In [6]:
# open spectra library
shoyu_data_dict = pickle.load(open('../raman_spectra/shoyu_data_dict.p', 'rb'))
# list keys
sorted(shoyu_data_dict.keys())

['2-PROPANOL',
 'CARBON DIOXIDE',
 'CARBON MONOXIDE',
 'DIMETHYL KETONE',
 'ETHYL ALCOHOL',
 'FORMALDEHYDE',
 'FORMIC ACID',
 'WATER']

In [ ]:
# extract spectra data
data = shoyu_data_dict['WATER']
x_data = data['x']
y_data = data['y']

data_range = max(x_data) - min(x_data)
data_range

In [ ]:
# subtract baseline
# y_data = spectrafit.subtract_baseline(y_data, deg=3, plot=True, x_data=x_data)

In [ ]:
# detect peaks
peaks = spectrafit.find_peaks(x_data, y_data, thres=0.25, min_dist=50)
# from operator import itemgetter
# peaks = sorted(peaks, key=itemgetter(1), reverse=True)[:5]
peaks

In [ ]:
fig = plt.figure(figsize=(15,6))
plt.plot(x_data, y_data)
for i in range(len(peaks)):
    plt.axvline(x=peaks[i][0], color='orange')

In [ ]:
# assign parameters for least squares fit
mod, pars = spectrafit.lorentz_params(peaks)

In [ ]:
# fit the model to the data
out = spectrafit.model_fit(x_data, y_data, mod, pars, report=True)

In [ ]:
# plot fit results
spectrafit.plot_fit(x_data, y_data, out, plot_components=True)

In [ ]:
# export data in logical structure (see docstring)
fit_peak_data = spectrafit.export_fit_data(out)
fit_peak_data

In [ ]:
# print docstring which outlines data structure
spectrafit.export_fit_data.__doc__

In [ ]:
peak_loc = []
peak_sigma = []
for i in range(len(fit_peak_data)):
    loc = fit_peak_data[i][1]
    sigma = fit_peak_data[i][0]
    peak_loc.append(loc)
    peak_sigma.append(sigma)


In [7]:
water = shoyu_data_dict['WATER']
x_water = water['x']
y_water = water['y']

co = shoyu_data_dict['CARBON MONOXIDE']
x_co = co['x']
y_co = co['y']

type(x_water)

numpy.ndarray

In [11]:
compound_curve = np.concatenate(x_water, x_co)

AttributeError: 'dict' object has no attribute 'append'

In [16]:
from ramannoodles import water
from ramannoodles import carbonmonoxide

file already in raman_spectra folder
WATER loaded into the dictionary - shoyu_data_dict.p
file already in raman_spectra folder
CARBON MONOXIDE loaded into the dictionary - shoyu_data_dict.p
file already in raman_spectra folder
CARBON DIOXIDE loaded into the dictionary - shoyu_data_dict.p
file already in raman_spectra folder
FORMIC ACID loaded into the dictionary - shoyu_data_dict.p
file already in raman_spectra folder
2-PROPANOL loaded into the dictionary - shoyu_data_dict.p
file already in raman_spectra folder
ETHYL ALCOHOL loaded into the dictionary - shoyu_data_dict.p
file already in raman_spectra folder
DIMETHYL KETONE loaded into the dictionary - shoyu_data_dict.p


FileNotFoundError: [Errno 2] No such file or directory: 'raman_spectra/shoyu_data_dict.p'

In [17]:
result = water.water()

In [15]:
result

(388.677,
 3799.46,
 [284.6239131319124, 60.32966482798571, 278.50492691505036],
 [678.0782334453711, 1637.5543502609387, 3368.238938601593],
 [669.7769952837992, 123.30210972210591, 992.5161639078589])